In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24522
2,Huelva,Confirmados PDIA 14 días,413
3,Huelva,Tasa PDIA 14 días,"80,48015277588324"
4,Huelva,Confirmados PDIA 7 días,126
5,Huelva,Total Confirmados,24744
6,Huelva,Curados,15994
7,Huelva,Fallecidos,295


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24522.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6543.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5532 personas en los últimos 7 días 

Un positivo PCR cada 1370 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24522.0,126.0,413.0,513170.0,24.553267,80.480153,35.0
Huelva-Costa,14962.0,74.0,243.0,289548.0,25.557075,83.923909,22.0
Condado-Campiña,6838.0,49.0,153.0,156231.0,31.363814,97.931909,13.0
Huelva (capital),6543.0,26.0,105.0,143837.0,18.076017,72.999298,8.0
Almonte,853.0,18.0,39.0,24507.0,73.448402,159.138205,5.0
Cartaya,937.0,14.0,36.0,20083.0,69.710701,179.256087,3.0
Isla Cristina,1982.0,6.0,11.0,21393.0,28.046557,51.418688,3.0
Lepe,1989.0,9.0,33.0,27880.0,32.281205,118.364419,3.0
Palos de la Frontera,448.0,2.0,12.0,11742.0,17.032873,102.197241,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,174.0,2.0,10.0,2885.0,69.324090,346.620451,0.0
Villanueva de los Castillejos,150.0,2.0,6.0,2825.0,70.796460,212.389381,0.0
Cartaya,937.0,14.0,36.0,20083.0,69.710701,179.256087,3.0
Rosal de la Frontera,25.0,2.0,3.0,1697.0,117.855038,176.782557,0.0
Bollullos Par del Condado,606.0,7.0,23.0,14387.0,48.655036,159.866546,0.0
Almonte,853.0,18.0,39.0,24507.0,73.448402,159.138205,5.0
Villalba del Alcor,443.0,3.0,5.0,3366.0,89.126560,148.544266,0.0
Cabezas Rubias,29.0,1.0,1.0,706.0,141.643059,141.643059,0.0
Moguer,821.0,4.0,26.0,21867.0,18.292404,118.900627,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,309.0,1.0,9.0,7939.0,12.596045,113.364404,0.0,0.111111
Moguer,821.0,4.0,26.0,21867.0,18.292404,118.900627,1.0,0.153846
Palos de la Frontera,448.0,2.0,12.0,11742.0,17.032873,102.197241,3.0,0.166667
Trigueros,228.0,1.0,6.0,7862.0,12.719410,76.316459,0.0,0.166667
Villablanca,174.0,2.0,10.0,2885.0,69.324090,346.620451,0.0,0.200000
Gibraleón,509.0,1.0,5.0,12737.0,7.851142,39.255712,0.0,0.200000
Ayamonte,1186.0,3.0,14.0,21104.0,14.215315,66.338135,2.0,0.214286
Punta Umbría,657.0,3.0,14.0,15355.0,19.537610,91.175513,1.0,0.214286
Sierra de Huelva-Andévalo Central,2298.0,2.0,9.0,67391.0,2.967755,13.354899,0.0,0.222222
